In [16]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
import sklearn.pipeline as pipe
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA

In [2]:
#using pandas to import the original training_text and traning_variant
sms = pd.read_table('training_rare_text.txt', encoding= 'UTF8', sep="\\|\\|",header=None, engine='python', names=['cli_data'])
sms2= pd.read_table('training_variants.txt', encoding= 'UTF8', header=None,delimiter=',', engine='python', names=['gene','mutation','classification'])

In [3]:
sms.shape

(3321, 1)

In [4]:
#a glimpse of the type, doesn't matter
type(sms)

pandas.core.frame.DataFrame

In [5]:
sms2.shape

(3321, 3)

In [6]:
#the first 10 lines of the text file
sms.head(10)

,cli_data
0,Cyclin dependent kinases CDKs regulate variety...
1,Abstract Background Non small lung NSCLC heter...
2,Abstract Background Non small lung NSCLC heter...
3,Recent evidence demonstrated acquired uniparen...
4,Oncogenic monomeric Casitas B lineage lymphoma...
5,Oncogenic monomeric Casitas B lineage lymphoma...
6,Oncogenic monomeric Casitas B lineage lymphoma...
7,CBL negative regulator activated receptor tyro...
8,Abstract Juvenile myelomonocytic leukemia JMML...
9,Abstract Juvenile myelomonocytic leukemia JMML...


In [7]:
#first 10 lines of the variants file
sms2.head(10)

,gene,mutation,classification
0,FAM58A,Truncating Mutations,1
1,CBL,W802*,2
2,CBL,Q249E,2
3,CBL,N454D,3
4,CBL,L399V,4
5,CBL,V391I,4
6,CBL,V430M,5
7,CBL,Deletion,1
8,CBL,Y371H,4
9,CBL,C384R,4


In [8]:
#the classification is very imbalanced use this dictinaty as the "class-weight" parameter when do model fitting
weighted=sms2.classification.value_counts()
weight_dict=dict()
for key in weighted.keys():
    weight_dict[key]=weighted[key]/3311
print(weight_dict)

{7: 0.2878284506191483, 4: 0.20718816067653276, 1: 0.17154938085170643, 2: 0.13651464814255512, 6: 0.083056478405315617, 5: 0.073089700996677748, 3: 0.026880096647538508, 9: 0.01117487163998792, 8: 0.0057384475989127152}


In [9]:
#define the X and y to be feed to the model
X = sms.cli_data
y = sms2.classification

In [10]:
#split the training data into "train" and "test" when run the model
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2490,)
(831,)
(2490,)
(831,)


In [6]:
#using kfold to split the data
kf=KFold(n_splits=3)
array_nb=[]
array_lr=[]
array_svc=[]
score_nb=0
score_lr=0
score_svc=0
i=0
for train_indices,test_indices in kf.split(X,y):
    X_train=X[train_indices]
    y_train=y[train_indices] 
    X_test=X[test_indices] 
    y_test=y[test_indices] 
    print(len(X_train))
    
    vect = CountVectorizer()
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)
    
    weighted=y_train.value_counts()
    weight_dict=dict()
    for key in weighted.keys():
        weight_dict[key]=weighted[key]/len(X_train)
    
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class_nb = nb.predict(X_test_dtm)
    score_nb+=metrics.accuracy_score(y_test, y_pred_class_nb)
    array_nb.append(metrics.accuracy_score(y_test, y_pred_class_nb))
    
    logreg = LogisticRegression(class_weight=weight_dict)
    logreg.fit(X_train_dtm, y_train)
    y_pred_class_lr = logreg.predict(X_test_dtm)
    score_lr+=metrics.accuracy_score(y_test, y_pred_class_lr)
    array_lr.append(metrics.accuracy_score(y_test, y_pred_class_lr))
    
    i=i+1



    

2214
2214
2214


In [7]:

print(score_nb)
print(score_lr)
print(score_svc)
print(i)
print(array_nb)
print(array_lr)
print(array_svc)

1.05962059621
1.10840108401
0
3
[0.32971996386630531, 0.4065040650406504, 0.32339656729900634]
[0.34869015356820232, 0.35953026196928634, 0.40018066847335138]
[]


In [9]:
accuracy_nb=score_nb/i
accuracy_lr=score_lr/i
print(accuracy_nb)
print(accuracy_lr)


0.353206865402
0.369467028004


In [ ]:
#clf = pipe.make_pipeline(TfidfVectorizer(), MultinomialNB())
#clf.fit_tansform(X,y)
#score=cross_val_score(clf, X, y, cv=10, scoring='accuracy')
#print(score)

In [11]:
#create the vectorized instance
vect = CountVectorizer()

In [12]:
#fit--learn the train vocabulary, as "features";transform is to create a "document term matrix"
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

<2490x137273 sparse matrix of type '<class 'numpy.int64'>'
	with 4129644 stored elements in Compressed Sparse Row format>

In [19]:
ipca = IncrementalPCA()
ipca.fit(X_train_dtm.toarray())

IncrementalPCA(batch_size=None, copy=True, n_components=None, whiten=False)

In [20]:
ipca.transform(X_train_dtm.toarray())

array([[ -5.84744804e+01,  -2.09821011e+00,   4.50863496e+01, ...,
          3.92914867e-15,  -3.82934786e-15,   1.10328413e-15],
       [ -7.15468658e+01,  -4.63838625e-01,   4.77686900e+01, ...,
          3.28070904e-14,   4.22289970e-15,   3.11556336e-14],
       [  2.39827948e+01,   5.74623663e+01,  -8.80535067e+01, ...,
          2.75474088e-15,  -8.80372164e-16,   4.65599781e-15],
       ..., 
       [ -7.24824237e+01,   4.12612782e+00,   2.64805798e+01, ...,
         -3.48505946e-15,   3.01488164e-15,   3.64985819e-15],
       [  1.11020892e+02,   1.71829060e+02,   6.36967139e+01, ...,
          2.17128399e-13,  -1.25868066e-13,  -1.35239042e-13],
       [  1.25748961e+01,  -1.37090070e+01,  -1.53634052e+01, ...,
         -7.09744763e-14,  -7.91276766e-14,   1.90117019e-14]])

In [9]:
#do the same thing to test file 
X_test_dtm = vect.transform(X_test)
X_test_dtm 

<831x137273 sparse matrix of type '<class 'numpy.int64'>'
	with 1290716 stored elements in Compressed Sparse Row format>

In [10]:
#use naive bayes model to test
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [11]:
#accuracy using cross-validation
y_pred_class_nb = nb.predict(X_test_dtm)
metrics.accuracy_score(y_test, y_pred_class_nb)

0.58604091456077012

In [12]:
#the first 10 lines of  prediction
print(y_pred_class_nb[:10])

[1 7 1 7 4 1 1 2 4 1]


In [13]:
metrics.confusion_matrix(y_test, y_pred_class_nb)
#class 2&7 1&4 are easy to get confused

array([[ 87,   7,   0,  16,  18,   5,   9,   0,   0],
       [  5,  52,   0,   5,   1,   0,  54,   0,   0],
       [  4,   1,  11,   2,   3,   1,   5,   0,   0],
       [ 54,   2,   1,  88,  16,   3,   5,   0,   1],
       [  9,   1,   0,   3,  24,   4,   8,   0,   1],
       [ 10,   5,   1,   0,   6,  35,   4,   0,   0],
       [  4,  55,   5,   1,   4,   0, 181,   0,   0],
       [  1,   1,   0,   0,   0,   0,   1,   1,   1],
       [  0,   0,   0,   0,   0,   0,   1,   0,   8]])

In [15]:
#first 10 lines of classification prediction probality 
y_pred_prob = nb.predict_proba(X_test_dtm)[:10]
y_pred_prob

array([[  1.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          1.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  1.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  1.70865906e-162,   2.75034046e-075,   0.00000000e+000,
          6.35541307e-163,   0.00000000e+000,   0.00000000e+000,
          1.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  5.83630847e-011,   0.00000000e+000,   0.00000000e+000,
          1.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  1.00000000

In [16]:
#use logistic regression model to test
logreg = LogisticRegression(class_weight=weight_dict)
logreg.fit(X_train_dtm, y_train)

LogisticRegression(C=1.0,
          class_weight={7: 0.2878284506191483, 4: 0.20718816067653276, 1: 0.17154938085170643, 2: 0.13651464814255512, 6: 0.083056478405315617, 5: 0.073089700996677748, 3: 0.026880096647538508, 9: 0.01117487163998792, 8: 0.0057384475989127152},
          dual=False, fit_intercept=True, intercept_scaling=1,
          max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2',
          random_state=None, solver='liblinear', tol=0.0001, verbose=0,
          warm_start=False)

In [17]:
y_pred_class = logreg.predict(X_test_dtm)

In [18]:
#predicted probiblity of each class for the first 10 lines
y_pred_prob_lr = logreg.predict_proba(X_test_dtm)[:10]
print(y_pred_prob_lr)

[[  9.99988442e-01   1.06558855e-11   8.30291578e-21   2.09112728e-10
    9.52141920e-13   1.15518796e-05   5.65273130e-09   2.41168829e-21
    7.84261402e-24]
 [  2.61632222e-09   1.17640396e-07   1.88508002e-04   1.40052587e-09
    2.97406845e-05   3.60008739e-09   9.99781620e-01   6.29421053e-09
    9.50902008e-12]
 [  9.99826533e-01   9.35293678e-07   4.49139471e-07   3.80754072e-05
    1.63223354e-05   5.68374735e-05   6.08423382e-05   5.07394926e-09
    8.69378227e-11]
 [  1.24193972e-03   3.72897207e-01   2.32254519e-05   3.95197289e-01
    1.76435491e-03   3.14284789e-02   1.97434720e-01   7.63474978e-08
    1.27079280e-05]
 [  2.11958262e-01   9.64285247e-16   1.17606231e-09   7.88041370e-01
    3.22433640e-07   4.36960682e-08   7.58629771e-10   5.86208226e-15
    1.85229237e-13]
 [  1.77231855e-02   2.22291307e-02   5.00531528e-03   8.21416501e-01
    2.10611042e-02   1.06138708e-01   5.25225164e-03   2.49227820e-04
    9.24575127e-04]
 [  2.67100116e-04   3.75715637e-10   9.

In [19]:
#the accuracy using log_regression model
metrics.accuracy_score(y_test, y_pred_class)

0.63297232250300839

In [20]:
#the inaccuray mainly happened between class 1&4, 2&7
metrics.confusion_matrix(y_test, y_pred_class)

array([[ 82,   3,   0,  36,   6,   7,   8,   0,   0],
       [  5,  57,   0,   4,   0,   4,  47,   0,   0],
       [  1,   0,   1,  10,   1,   0,  14,   0,   0],
       [ 24,   1,   1, 125,  12,   2,   5,   0,   0],
       [ 15,   0,   0,   9,  10,   5,  11,   0,   0],
       [  7,   5,   1,   2,   3,  40,   3,   0,   0],
       [  5,  31,   0,   5,   5,   2, 202,   0,   0],
       [  0,   0,   0,   0,   0,   0,   2,   3,   0],
       [  0,   0,   0,   2,   0,   0,   1,   0,   6]])

In [21]:
svc=SVC(class_weight=weight_dict)
svc.fit(X_train_dtm,y_train)

SVC(C=1.0, cache_size=200,
  class_weight={7: 0.2878284506191483, 4: 0.20718816067653276, 1: 0.17154938085170643, 2: 0.13651464814255512, 6: 0.083056478405315617, 5: 0.073089700996677748, 3: 0.026880096647538508, 9: 0.01117487163998792, 8: 0.0057384475989127152},
  coef0=0.0, decision_function_shape='ovr', degree=3, gamma='auto',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [22]:
y_pred_class_svc=svc.predict(X_test_dtm)

In [23]:
#the accuary of svc model is the lowest
metrics.accuracy_score(y_test, y_pred_class_svc)

0.46089049338146809

In [ ]:
#use KNN model to test

In [24]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_dtm, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [25]:
y_pred_class_knn= knn.predict(X_test_dtm)

In [26]:
#get the accuracy using knn model
metrics.accuracy_score(y_test, y_pred_class_knn)

0.59687123947051746